In [1]:
%%capture
!pip install -U bitsandbytes

In [22]:
!pip install transformers -U
!pip install --upgrade pip setuptools wheel
!pip install -U bitsandbytes
!pip install flash-attn --no-build-isolation
!pip install accelerate -U
!pip install --upgrade huggingface-hub==0.24.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 43.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [2]:
import torch
import warnings
import os
from collections import defaultdict
import numpy as np
import pandas as pd
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from transformers import logging
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

logging.set_verbosity_error()
warnings.filterwarnings("ignore")

In [9]:
MODEL_NAME = "codellama/CodeLlama-13b-Instruct-hf"

QUANTIZATION_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

MODEL = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    quantization_config=QUANTIZATION_CONFIG,
    device_map="auto",
    attn_implementation = "flash_attention_2"
)

TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
TOKENIZER.use_default_system_prompt = False

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [4]:
GENERATION_CONFIG = GenerationConfig(
    max_new_tokens=256,
    # do_sample=False,
    num_beams=2,
    #num_beam_groups=2,
    # diversity_penalty=1.0,

    # num_return_sequences=2,
    eos_token_id=TOKENIZER.eos_token_id,
)

In [5]:
code = '''
import random

def generate_random_number():
    """Generate a random number between 1 and 100."""
    return random.randint(1, 100)

def calculate_square(num):
    """Return the square of a number."""
    return num ** 2

def calculate_square_root(num):
    """Return the square root of a number."""
    if num < 0:
        raise ValueError("Cannot calculate square root of a negative number.")
    return num ** 0.5

def print_square(num):
    """Print the square of a number."""
    square = calculate_square(num)
    print(f"The square of {num} is {square}.")

def print_square_root(num):
    """Print the square root of a number."""
    square_root = calculate_square_root(num)
    print(f"The square root of {num} is {square_root}.")

def get_user_input():
    """Get user input and return as an integer."""
    user_input = 10
    return int(user_input)

def main():
    """Main function to execute the program."""
    try:
        number = get_user_input()
        print_square(number)
        print_square_root(number)

        # Intentional error: calling an undefined function
        undefined_function()  # This will cause a NameError

        random_number = generate_random_number()
        print(f"A random number generated: {random_number}")
        print_square(random_number)
        print_square_root(random_number)

    except ValueError as ve:
        print(f"Value error occurred: {ve}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Unused function
def unused_function():
    """This function is not used anywhere in the code."""
    print("This function is never called.")

# Another unused function
def another_unused_function():
    """Another unused function."""
    print("This function is also never called.")

# Additional functions to demonstrate complexity
def factorial(n):
    """Return the factorial of a number."""
    if n < 0:
        raise ValueError("Cannot calculate factorial of a negative number.")
    if n == 0 or n == 1:
        return 1
    return n * factorial(n - 1)

def print_factorial(num):
    """Print the factorial of a number."""
    fact = factorial(num)
    print(f"The factorial of {num} is {fact}.")

def fibonacci(n):
    """Return the nth Fibonacci number."""
    if n < 0:
        raise ValueError("Cannot calculate Fibonacci of a negative number.")
    elif n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

def print_fibonacci(n):
    """Print Fibonacci sequence up to n."""
    fib_seq = [fibonacci(i) for i in range(n)]
    print(f"Fibonacci sequence up to {n}: {fib_seq}")

def run_additional_operations():
    """Run some additional operations."""
    num = generate_random_number()
    print_square(num)
    print_fibonacci(10)

if __name__ == "__main__":
    main()
    run_additional_operations()
'''
problem_part = """
        undefined_function()
"""

In [6]:
import ast
import re
import sys

sys.setrecursionlimit(2000)

def find_function_body(tree, search_line, source_code):
    def get_function_by_name(func_name, visited):
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef) and node.name == func_name and func_name not in visited:
                visited.add(func_name)
                return node
        return None

    def iterative_find_function_body(node, collected_functions, visited):
        stack = [node]
        while stack:
            current_node = stack.pop()
            function_body = ast.get_source_segment(source_code, current_node)
            collected_functions.append(function_body)

            for child in ast.walk(current_node):
                if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                    func_name = child.func.id
                    called_function_node = get_function_by_name(func_name, visited)
                    if called_function_node:
                        stack.append(called_function_node)

    collected_code = []
    global_statements = [node for node in tree.body if not isinstance(node, ast.FunctionDef)]

    visited = set()

    for node in global_statements:
        global_code = ast.get_source_segment(source_code, node)
        if search_line in global_code:
            collected_code.append(global_code)

        for child in ast.walk(node):
            if isinstance(child, ast.Call) and isinstance(child.func, ast.Name):
                func_name = child.func.id
                called_function_node = get_function_by_name(func_name, visited)
                if called_function_node:
                    iterative_find_function_body(called_function_node, collected_code, visited)

    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            function_body = ast.get_source_segment(source_code, node)
            if search_line in function_body:
                iterative_find_function_body(node, collected_code, visited)

    return "\n\n".join(collected_code) if collected_code else None


def find_args(funcsource):
    funcs = []
    for signature in re.findall(r'def .+:', funcsource):
        args = re.findall(
            r'\(.*\)', signature)[0].replace('(', '').replace(')', '').split(',')
        args = [arg.strip() for arg in args]
        funcs.append((signature.strip(), args))
    return funcs


def get_imports(code):
    """Extracts import statements from the given code."""
    import_pattern = r'^\s*(import\s+\S+|from\s+\S+\s+import\s+\S+)'
    imports = re.findall(import_pattern, code, re.MULTILINE)
    return imports


def generate_prompt(code_string, problem_part):
    tree = ast.parse(code_string)
    needed_context = find_function_body(tree, problem_part, code_string)
    prompt = ''
    imports = get_imports(code_string)
    imports = ''.join(imports)
    variable_pattern = r"(\w+)\s*=\s*(.+)"
    variables = re.findall(variable_pattern, code_string)
    variable_types = {}
    vars = []
    for var_name, value in variables:
        if value.strip() in variable_types:
            inferred_type = variable_types[value.strip()]
        else:
            try:
                inferred_type = eval(f'type({value.strip()})')
            except Exception:
                inferred_type = 'unknown'

        variable_types[var_name] = inferred_type
    for var_name, inferred_type in variable_types.items():
        vars.append(f"Variable Name: {var_name}, Inferred Type: {inferred_type.__name__ if hasattr(inferred_type, '__name__') else inferred_type}")
    vars = '\n'.join(vars)
    prompt = '\n'.join(['Imports: ', imports,
                       'Original code:', needed_context,
                       'Variables in code:',vars,
                       'Part needs explanation:', problem_part,
                       'Documentation for used functions:', ])
    print(prompt)

def llm_generate_prompt(source_code, trouble_code):
    system_message = {
        "role": "system",
        "content": """\nYou're a specialized AI assisting with solving problems of troublesome code on Python.
          For every right recommendation you get 500 bucks. My job depends on it.
          Answer in a following manner: 1. Line 23 has an unidentified variable
        or  1. Line 101 calls a function with inappropriate arguments.
          NEVER give the right code as answer, only reasons why it doesn't work.\n""",
    }

    prompt = {
        "role": "user",
        "content": (
            f'{generate_prompt(source_code, problem_part="cool")}. Explain me the reason why it does not work and add links to documentation for troublesome functions if needed.'
        ),
    }
    return [system_message, prompt]

# Example usage
# source_code = your_code_here
# trouble_code = part_of_code_where_issue_is


In [8]:
messages =llm_generate_prompt(code, problem_part)

inputs = TOKENIZER.apply_chat_template(
    messages, add_generation_prompt=True, return_tensors="pt"
).to(MODEL.device)
outputs = MODEL.generate(
    inputs,
    generation_config=GENERATION_CONFIG,
)

response = [TOKENIZER.decode(
    output[len(inputs[0]) :], skip_special_tokens=True
).strip() for output in outputs]

Imports: 
import random
Original code:
def main():
    """Main function to execute the program."""
    try:
        number = get_user_input()
        print_square(number)
        print_square_root(number)

        # Intentional error: calling an undefined function
        undefined_function()  # This will cause a NameError

        random_number = generate_random_number()
        print(f"A random number generated: {random_number}")
        print_square(random_number)
        print_square_root(random_number)

    except ValueError as ve:
        print(f"Value error occurred: {ve}")
    except Exception as e:
        print(f"An error occurred: {e}")

def generate_random_number():
    """Generate a random number between 1 and 100."""
    return random.randint(1, 100)

def print_square_root(num):
    """Print the square root of a number."""
    square_root = calculate_square_root(num)
    print(f"The square root of {num} is {square_root}.")

def calculate_square_root(num):
    """Return th

RuntimeError: FlashAttention only supports Ampere GPUs or newer.

In [ ]:
print(response[0])